## Import libraries

In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import spikeinterface.full as si
from probeinterface import read_probeinterface
from probeinterface.plotting import plot_probe

%matplotlib widget

In [3]:
print(f"SpikeInterface version: {si.__version__}")

SpikeInterface version: 0.99.0.dev0


## Define paths and configure default parameters

In [ ]:
# Main working directory
base_folder = Path.home() / 'RANCZLAB-NAS' / 'iakov' / 'allen_sorting_07_11_23'

# Input data
binary_filename = 'spike_band.dat'

# Output sorting
output_folder = base_folder / 'kilosort3_sorting_files'
sorting_save_path = base_folder / 'kilosort3_sorting_output'

In [ ]:
# Default parameters for compute-intensive steps
# n_jobs is the number of CPU cores used. -1 means use all cores.
job_kwargs = dict(n_jobs=-1, chunk_duration="1s", progress_bar=True)

# Plotting backend
backend = 'ipywidgets'

## Load data

In [ ]:
# Recording parameters
sampling_frequency = 32000
dtype = np.int16
channel_number = 384

In [ ]:
recordings = {}
recordings['raw'] = si.read_binary(base_folder / binary_filename, sampling_frequency=sampling_frequency, dtype=dtype, num_chan=channel_number)

## Assign probe

In [ ]:
probegroup = read_probeinterface('np_json_files/np1.json')
probe = probegroup.probes[0]
recordings['raw'] = recordings['raw'].set_probe(probe)

plot_probe(probe)

## Apply preprocessing

### Bandpass filtering

In [ ]:
recordings['bandpass'] = si.bandpass_filter(recordings['raw'], freq_min=300, freq_max=9000)

### Common median reference

In [ ]:
recordings['common_median'] = si.common_reference(recordings['bandpass'], reference='global', operator='median')

## Plot data

In [ ]:
si.plot_traces(recordings, mode='map', time_range=[0, 10], backend=backend)

## Save preprocessed recording

In [ ]:
recordings['common_median'].save(folder=base_folder / "preprocessed_recording", **job_kwargs)

## Send to sorting

### Define parameters

In [15]:
print(f"List of available spike sorters (in Singularity containers):\n{os.listdir(Path.home() / 'RANCZLAB-NAS' / 'spikesorter_images')}")

List of available spike sorters (in Singularity containers):
['kilosort-compiled-base.sif', 'kilosort3-compiled-base.sif', 'mountainsort5-base.sif', 'hdsort-compiled-base.sif', 'ironclust-compiled-base.sif']


In [ ]:
sorter_name = 'kilosort3'
sorter_path = Path.home() / 'RANCZLAB-NAS' / 'spikesorter_images' / 'kilosort3-compiled-base.sif'

### Launch

In [ ]:
sorting = si.run_sorter(sorter_name, recording=recordings['common_median'], output_folder=output_folder, singularity_image=sorter_path, verbose=True, **job_kwargs)
sorting.save(folder=sorting_save_path, format='npz_folder', **job_kwargs)